# Data preparation 
---
Avant importation, les bases de données ont été nettoyées de la manière suivante : 
- Suppression des colonnes inutiles dans le jeu athlete_events.csv, pour ne garder que les médailles, les années, les villes et les codes pays, ainsi que la distinction entre hiver et été pour pouvoir ne garder que le subset des JO d'été 

In [2]:
# importation du fichier et des packages 
import pandas as pd 
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.read_csv("./datasets_clean/olympics_main/socio_athletes.csv", sep = ";")
# on ne garde que les JO d'été et on supprime la colonne saison ensuite
df = df.loc[df["Season"].str.contains("Summer")]
df = df.drop(columns = "Season")
# on groupe par année, médailles, et pays  
athlete_series = df.groupby(["NOC", "Year", "Medal"]).size()
df = athlete_series.to_frame(name = "Number of Medals").reset_index()
display(df)
# Pivot the dataframe to have separate columns for each medal type
df_pivot = df.pivot(index=["NOC", "Year"], columns="Medal", values="Number of Medals").reset_index().fillna(0)
df_pivot["Total"] = df_pivot["Bronze"] + df_pivot["Gold"] + df_pivot["Silver"]
display(df_pivot)
df_pivot.to_csv("./datasets_clean/olympics_main/country_year_medals.csv")

,NOC,Year,Medal,Number of Medals
0,AFG,2008,Bronze,1
1,AFG,2012,Bronze,1
2,AHO,1988,Silver,1
3,ALG,1984,Bronze,2
4,ALG,1992,Bronze,1
...,...,...,...,...
2804,ZIM,2004,Bronze,1
2805,ZIM,2004,Gold,1
2806,ZIM,2004,Silver,1
2807,ZIM,2008,Gold,1


Medal,NOC,Year,Bronze,Gold,Silver,Total
0,AFG,2008,1.0,0.0,0.0,1.0
1,AFG,2012,1.0,0.0,0.0,1.0
2,AHO,1988,0.0,0.0,1.0,1.0
3,ALG,1984,2.0,0.0,0.0,2.0
4,ALG,1992,1.0,1.0,0.0,2.0
...,...,...,...,...,...,...
1270,ZAM,1984,1.0,0.0,0.0,1.0
1271,ZAM,1996,0.0,0.0,1.0,1.0
1272,ZIM,1980,0.0,15.0,0.0,15.0
1273,ZIM,2004,1.0,1.0,1.0,3.0


In [33]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# pivoter table PIB 
gdp_cap = pd.read_csv("../../Datasets_raw/indicators/GDP/GDP_raw.csv", on_bad_lines= "warn" )
gdp_cap = gdp_cap.drop(columns = ["Indicator Code", "Unnamed: 67"])
gdp_pivot = gdp_cap.melt(id_vars=["Country Name", "Country Code"], value_vars=gdp_cap.columns[3:-1], var_name="Year", value_name="GDP")
gdp_pivot = gdp_pivot.dropna()
gdp_pivot['GDP'] = gdp_pivot['GDP'].astype(float)/1000000
gdp_pivot.to_csv("../../dataprep/David/datasets_clean/gdp_country.csv")

display(gdp_pivot)

Skipping line 25: expected 68 fields, saw 69
Skipping line 45: expected 68 fields, saw 69
Skipping line 46: expected 68 fields, saw 69
Skipping line 69: expected 68 fields, saw 69
Skipping line 81: expected 68 fields, saw 69
Skipping line 88: expected 68 fields, saw 69
Skipping line 98: expected 68 fields, saw 69
Skipping line 114: expected 68 fields, saw 69
Skipping line 128: expected 68 fields, saw 69
Skipping line 148: expected 68 fields, saw 69
Skipping line 195: expected 68 fields, saw 69
Skipping line 256: expected 68 fields, saw 69
Skipping line 264: expected 68 fields, saw 69



,Country Name,Country Code,Year,GDP
1,Africa Eastern and Southern,AFE,1960,153696.396
3,Africa Western and Central,AFW,1960,105675.520
9,Argentina,ARG,1960,150797.810
13,Australia,AUS,1960,205000.838
14,Austria,AUT,1960,84930.958
...,...,...,...,...
15681,Samoa,WSM,2021,843.469
15682,Kosovo,XKX,2021,7912.074
15683,South Africa,ZAF,2021,353258.433
15684,Zambia,ZMB,2021,24495.921


In [46]:
# filtrer table IDH pour pas faire peur à Tableau et aussi la pivoter
hdi = pd.read_csv("../../Datasets_raw/indicators/HDI/HDR21-22_Composite_indices_complete_time_series.csv")
hdi = hdi.melt(id_vars=["iso3", "country"], value_vars=hdi.columns[4:-1], var_name="indicator_year", value_name="indicator_value")
hdi = hdi.loc[hdi["indicator_year"].str.contains("hdi|gdi|ihdi|gii|phdi")]
hdi = hdi[hdi["indicator_year"].str.contains("rank|diff|group|_f|_m") == False]
hdi.to_csv("../David/datasets_clean/hdi_clean.csv")


In [54]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# pivoter population 
pop_country = pd.read_csv("../../Datasets_raw/indicators/demography/pop_year.csv", on_bad_lines="warn")
pop_country = pop_country.drop(columns = ["Indicator Code", "Indicator Name", "Unnamed: 67"])
pop_country = pop_country.melt(id_vars=["Country Name", "Country Code"], value_vars=pop_country.columns[2:-1], var_name="Year", value_name="population (millions)")
pop_country = pop_country.dropna()
pop_country['population (millions)'] = pop_country['population (millions)'].astype(float)/1000000
pop_country = pop_country.drop(columns = "Country Code")
pop_country.to_csv("../David/datasets_clean/country_poop_clean.csv")

Skipping line 25: expected 69 fields, saw 70
Skipping line 45: expected 69 fields, saw 70
Skipping line 46: expected 69 fields, saw 70
Skipping line 69: expected 69 fields, saw 70
Skipping line 81: expected 69 fields, saw 70
Skipping line 88: expected 69 fields, saw 70
Skipping line 98: expected 69 fields, saw 70
Skipping line 114: expected 69 fields, saw 70
Skipping line 128: expected 69 fields, saw 70
Skipping line 148: expected 69 fields, saw 70
Skipping line 195: expected 69 fields, saw 70
Skipping line 256: expected 69 fields, saw 70
Skipping line 264: expected 69 fields, saw 70

